In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("tweets2020.csv",index_col="Unnamed: 0")

In [3]:
df.head()

,2,Classificação
0,RT @AjudemSerena: IMPLORO POR SOCORRO!\nPor fa...,0
1,@ShakedownVail Yo. What a #kickass #chill bar ...,0
2,"RT @grazianopascale: ""Parece la trama de un fi...",0
3,RT @marcofeliciano: Pare e compare: PETROBRAS ...,2
4,RT @NetoItirussu: Hoje em depoimento da operaç...,0


### Link pra uso do bard:
https://ai.google.dev/gemini-api/docs/get-started/python?hl=pt-br

In [4]:
#pip install -q -U google-generativeai

In [5]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

/media/marcos/2caebd18-4175-48de-967d-932bd88604d32/Ajuda Mestrado/tweets/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
#GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
GOOGLE_API_KEY = ""
genai.configure(api_key=GOOGLE_API_KEY)

In [7]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [8]:
model = genai.GenerativeModel('gemini-pro')

In [9]:
"""
Classificação de cada tweet:
0- neutro, 1 - positivo, 2- negativo, 3- tweets que não são relacionados à bolsa, 10- Tweets não classificados

"""


def classify_tweet(tweet):
    question = "Estou classificando tweets em português brasileiro sobre a bolsa de valores do brasil. Quero que você classifique esse tweet em sentimentos onde cada número é um sentimento: 0- neutro, 1 - positivo, 2- negativo, 3- tweets que não são relacionados à bolsa. Esse é o tweet: "+tweet+". Me mande apenas o número da classificação"
    response = model.generate_content(question)
    response = response.text
    print(response)
    if '0' in response.split():
        return 0
    elif '1' in response.split():
        return 1
    elif '2' in response.split():
        return 2
    
    else:
        return None
    


In [10]:
df.head(71)

,2,Classificação
0,RT @AjudemSerena: IMPLORO POR SOCORRO!\nPor fa...,0
1,@ShakedownVail Yo. What a #kickass #chill bar ...,0
2,"RT @grazianopascale: ""Parece la trama de un fi...",0
3,RT @marcofeliciano: Pare e compare: PETROBRAS ...,2
4,RT @NetoItirussu: Hoje em depoimento da operaç...,0
...,...,...
66,@vitorino_gyn @UOLNoticias 13 anos PT FEZ NDA ...,10
67,B2W Digital recruta profissionais para vagas d...,10
68,RT @PRV10Z1TERMINA2: Não as privatizações da C...,10
69,@parafrentebrasi Eis pq a Petrobras quebrou. T...,10


1,1 segundo para cada 1 tweet

In [11]:
last_index = 0
count = 0

# Selecionar índices que são maiores ou iguais a last_index
indices = df.index[df.index >= last_index]

for index in indices:
    if count > 5:
        break
    
    try:
        # Verificar se a classificação atual é 10
        if df.loc[index, 'Classificação'] == 10:
            count += 1
            print(f"Processando índice: {index}")
            tweet = df.loc[index, '2']
            response = classify_tweet(tweet)

            if response is not None:
                df.loc[index, 'Classificação'] = int(response)
                print(f"Índice {index} atualizado com a classificação {response}. Count atualizado: {count}")
            else:
                print(f"Response é None para o índice {index}, continuando...")

    except Exception as e:
        print(f"Erro ao processar o índice {index}: {e}")
        continue


Processando índice: 61
Erro ao processar o índice 61: 429 Resource has been exhausted (e.g. check quota).
Processando índice: 62
Erro ao processar o índice 62: 429 Resource has been exhausted (e.g. check quota).
Processando índice: 63
Erro ao processar o índice 63: 429 Resource has been exhausted (e.g. check quota).
Processando índice: 64
Erro ao processar o índice 64: 429 Resource has been exhausted (e.g. check quota).
Processando índice: 65
Erro ao processar o índice 65: 429 Resource has been exhausted (e.g. check quota).
Processando índice: 66
Erro ao processar o índice 66: 429 Resource has been exhausted (e.g. check quota).
                                                   2  Classificação
0  RT @AjudemSerena: IMPLORO POR SOCORRO!\nPor fa...              0
1  @ShakedownVail Yo. What a #kickass #chill bar ...              0
2  RT @grazianopascale: "Parece la trama de un fi...              0
3  RT @marcofeliciano: Pare e compare: PETROBRAS ...              2
4  RT @NetoItirussu: Hoj

In [ ]:
df.head(71)

In [ ]:
distribuicao = df['Classificação'].value_counts()
print(distribuicao)


In [ ]:
df.head(12290)
print(len(df))

In [ ]:
def rate_tweets(df, last_index):
    indices = df.index[df.index >= last_index]

    for index in indices:
        print(df.loc[index, :])
        answer = int(input("Coloque 0 para continuar, 1 para alterar classificação e 2 para parar: "))
        if answer == 0:
            continue
        elif answer == 1:
            right_answer = int(input("Coloque a classificação correta: "))
            df.loc[index, "Classificação"] = right_answer
            print(len(df[df["Classificação"] == right_answer]))  # Corrigido
        else:
            return df  # Corrigido
        
    return df


In [ ]:
pd.set_option('display.max_rows', None)  # Mostra todas as linhas
pd.set_option('display.max_columns', None)  # Mostra todas as colunas
pd.set_option('display.width', None)  # Largura da exibição
pd.set_option('display.max_colwidth', None)  # Mostra todo o conteúdo das células

df_test = rate_tweets(df,0)

In [ ]:
df_test.head() 

In [ ]:
df.to_csv("tweets2020.csv")